In [8]:
from datetime import datetime
from datetime import timedelta
import yfinance as yf
import time
import math
import pandas as pd
import pandas_ta as ta
import json
import numpy as np
import pandas as pd
import http.client
import json
#casove pasma
import pytz
import os

In [9]:
#get current date and time
tz_BA = pytz.timezone('Europe/Bratislava')
current_datetime = datetime.now(tz_BA).strftime("%Y-%m-%d %H-%M-%S")
# convert datetime obj to string
str_current_datetime = str(current_datetime)
# create a file name
file_name = "//home//jupyter-fiko//sesions log//"+"sesionBTCETH "+str_current_datetime+".txt"

#We can adjust Jupyter with just a pinch of code so that it saved the output directly to a file on the server
import sys
import logging
so = open(file_name, 'w', 10)
sys.stdout.echo = so
sys.stderr.echo = so
get_ipython().log.handlers[0].stream = so
get_ipython().log.setLevel(logging.INFO)

In [10]:
%run bot_functions.ipynb
%run capital_commands.ipynb

In [11]:
asset_list = [
    {'ticker':'BTC', 'yfticker':yf.Ticker('BTC-USD'),'capitalticker':'BTCUSD','apiticker':'Bitcoin / USD'},
    {'ticker':'ETH', 'yfticker':yf.Ticker('ETH-USD'),'capitalticker':'ETHUSD','apiticker':'Ethereum / USD'},
    {'ticker':'XRP', 'yfticker':yf.Ticker('XRP-USD'),'capitalticker':'ETHUSD','apiticker':'Ripple / USD'}
]

In [12]:
#acount ids
#190868445419885726 = tradingbot#192725185551750302 = daily rsi test

In [13]:
keys =get_demo_keys()

In [ ]:
#tradelog = []
position_size = 0.04
stop_loss = 0.005
while True:
    ping(keys)
    switch_acount(keys, 192725185551750302)
    #acount_balance_avaivable(keys,'daily rsi test') if u want balance without opened positions
    balance =  acount_balance(keys,'daily rsi test')
    for asset in asset_list:
        start_date = (datetime.now()-timedelta(days=2)).strftime('%Y-%m-%d')
        df = asset['yfticker'].history(start=start_date, interval='1m')
        price = df.iloc[-1]['Close']
        leverage = instrument_leverage(keys,asset['capitalticker'])
        position = round((position_size * balance * leverage) / price,2)
        df['rsi']  = RSI_Calc(df)
        tz_BA = pytz.timezone('Europe/Bratislava')
        BA = datetime.now(tz_BA)
        print( asset['ticker']+" current min. rsi = ",df.iloc[-1]['rsi']," , time = ", BA.strftime("%H:%M:%S") )
        if df.iloc[-2]['rsi'] < 50 and df.iloc[-1]['rsi'] > 50 :
            print(f"Buy {asset['ticker']} {price}")
            #tradelog.append({'date': datetime.now(), 'ticker': asset['yfticker'], 'side': 'buy', 'price': price})
            if get_id_from_instrument_name(keys,asset['apiticker']) != []:
                close_position(keys,get_id_from_instrument_name(keys,asset['apiticker']))
                print(f"closed previous postion {asset['ticker']} {price}")
            open_position_with_stoploss(keys, asset['capitalticker'], 
            "BUY", position,stop_loss_value(price, position , stop_loss, balance ,"BUY",instrument_leverage(keys,asset['capitalticker'])))
            
        elif df.iloc[-2]['rsi'] > 50 and df.iloc[-1]['rsi'] < 50:
            print(f"Sell {asset['ticker']} {price}")
            #tradelog.append({'date': datetime.now(), 'ticker': asset['yfticker'], 'side': 'sell', 'price': price})
            if get_id_from_instrument_name(keys,asset['apiticker']) != []:
                close_position(keys,get_id_from_instrument_name(keys,asset['apiticker']))
                print(f"closed previous postion {asset['ticker']} {price}")
            open_position_with_stoploss(keys, asset['capitalticker'], 
            "SELL", position,stop_loss_value(price, position , stop_loss, balance ,"SELL", instrument_leverage(keys,asset['capitalticker'])))
        
    time.sleep(get_pause())